In [1]:
import torch
import torch.nn as nn
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
parent_path = 'C:\\Users\\ismai\\Documents\\Comment-Section-UI\\angular-interactive-comments-section\\flask\\data'

In [2]:

file_name = 'fox-new-york-could-face'
file = os.path.join(parent_path, f'{file_name}.csv')
df = pd.read_csv(file)

In [3]:
df.head(5)

,Unnamed: 0,comment,time,username,id,timestamp,time_diff
0,321,And so ... the 'lawful' cheating begins!,"22 Dec, 2023 03:28 AM",senorbiggles,322,2023-12-22 03:28:00,0 days 00:00:00
1,875,And so ... the 'lawful' cheating begins!,"22 Dec, 2023 03:28 AM",senorbiggles,876,2023-12-22 03:28:00,0 days 00:00:00
2,296,Democrats desperately rigging the elections in...,"22 Dec, 2023 03:28 AM",HarryBaals404,297,2023-12-22 03:28:00,0 days 00:00:00
3,850,Democrats desperately rigging the elections in...,"22 Dec, 2023 03:28 AM",HarryBaals404,851,2023-12-22 03:28:00,0 days 00:00:00
4,144,Democrats can’t win on their achievements so t...,"22 Dec, 2023 03:29 AM",HarryBaals404,145,2023-12-22 03:29:00,0 days 00:01:00


In [4]:
list(df.iloc[1][1:])

["And so ... the 'lawful' cheating begins!",
 '22 Dec, 2023 03:28 AM',
 'senorbiggles',
 876,
 '2023-12-22 03:28:00',
 '0 days 00:00:00']

In [5]:
def get_so_far_comments(ith_comment):
    # row_index = df.loc[df['id'] == comment_id].index[0]
    ids = df.iloc[0:ith_comment+1]['id']
    return list(ids)


In [6]:
# get_so_far_comments(1)

In [7]:
df_trend = pd.read_csv(os.path.join(parent_path, f'{file_name}-trends.csv'))

In [8]:
df_trend.shape

(1064, 10)

In [9]:
def get_single_trend_score(trend):
    print("label: ", trend)
    return list(df_trend[trend])

In [10]:
# get_single_trend_score('toxic')

In [11]:
previous_emotion = []

In [12]:
# import random

# def get_emotion(escores, id):
#     # print("id")
#     if id == 0:
#         previous_emotion = escores

#     if id%100 == 0:
#         X = escores[27]*0.6
#         escores[27] = escores[27] - X
#         y = X*(random.sample(range(0, 100), 1)[0]/100)
#         random_numbers = random.sample(range(0, 27), 3)
#         escores[random_numbers[0]] = X*0.2
#         escores[random_numbers[1]] = X*0.7
#         escores[random_numbers[2]] = X*0.1
#         previous_emotion = escores
#         return escores
    
#     if id%200 == 0:
#         X = escores[27]*0.6
#         escores[27] = escores[27] - X
#         y = X*(random.sample(range(0, 100), 1)[0]/100)
#         random_numbers = random.sample(range(0, 27), 3)
#         escores[random_numbers[0]] = X*0.1
#         escores[random_numbers[1]] = X*0.6
#         escores[random_numbers[2]] = X*0.3
#         previous_emotion = escores
#         return escores
    
#     return previous_emotion


In [13]:
from common_function import get_emotions, get_sentiment_dict
from score_aggregation import get_polarity_score, find_indices

emotions = list(get_emotions().values())
sentiment_dict = get_sentiment_dict()

In [14]:
pos_indices = find_indices(sentiment_dict['positive'], emotions)
neg_indices = find_indices(sentiment_dict['negative'], emotions)
amb_indices = find_indices(sentiment_dict['ambiguous'], emotions)
neu_indices = find_indices(sentiment_dict['neutral'], emotions)

def get_emotion(escores, id):
    # print(escores)

    if id < 200:
        X = escores[27]*0.5
        escores[27] = escores[27] - X
        for i in amb_indices:
            escores[i] += (X*0.10)

    elif id > 200 and id < 500:
        X = escores[27]*0.6
        escores[27] = escores[27] - X
        for i in pos_indices:
            escores[i] += (X*0.10)
    elif id > 500:
        X = escores[27]*0.7
        escores[27] = escores[27] - X
        for i in neg_indices:
            escores[i] += (X*0.10)

    # print(escores)
    escores =  [float(i)/sum(escores) for i in escores]

    # print("SUM:", sum(escores))
    return escores

In [15]:
from score_aggregation import get_emotion_trends_polarity

def getData(index):
    ids = get_so_far_comments(ith_comment=index)
    print("Index: ", index)
    print("So far Comments: ", ids)
    escores, tscores, polarity = get_emotion_trends_polarity(ids)
    return escores, tscores, polarity

In [16]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json

app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

@app.route('/api/new-comment', methods=['POST'])
def welcome():
    comment = request.json['content']
    img = request.json['img']
    isYou = request.json['isYou']
    score = request.json['score']
    username = request.json['username']
    print("new: ", comment)
    return 'Successfully comment added!'

@app.route('/api/data/<int:id>', methods=['GET'])
def getApiData(id: int):
    escores, tscores, polarity = getData(id)
    escores = get_emotion(escores, id)
    polarity = get_polarity_score(escores)
    return json.dumps({"escores": escores, "tscores": tscores, "polarity": polarity})

@app.route('/api/data/single-trend/<string:label>', methods=['GET'])
def getSingleTrend(label: str):
    return json.dumps({"singletrend": get_single_trend_score(label)})

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2024-01-08 00:17:25,384] ERROR in app: Exception on /api/data/0 [GET]
Traceback (most recent call last):
  File "C:\Users\ismai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ismai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\ismai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "C:\Users\ismai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache

Index:  0
So far Comments:  [322]
